In [1]:
import dask, concurrent.futures, time, warnings, os, re, pickle
from osgeo import gdal
import os
import glob
import requests as r
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio as rio
import rioxarray as riox
import time
import xarray as xr
from urllib.request import urlopen
from xml.etree.ElementTree import parse,fromstring
from pandas import to_datetime
from rasterio.crs import CRS
from datetime import datetime, timedelta
from netrc import netrc
from pyproj import Proj
from hlsstack.hls_funcs.masks import mask_hls, shp2mask, bolton_mask_xr, atsa_mask
from hlsstack.hls_funcs.indices import ndvi_func
from hlsstack.hls_funcs.smooth import smooth_xr, despike_ts_xr
import cartopy.crs as ccrs
from rasterio.plot import show
from hlsstack.hls_funcs.predict import pred_bm, pred_bm_se, pred_cov
from hlsstack.models.load import load_model
import dask.diagnostics
import matplotlib.pyplot as plt
import gc


ERROR 1: PROJ: proj_create_from_database: Open of /project/cper_neon_aop/conda_envs/hls_nrt_env/share/proj failed


In [2]:
os.environ.update({'OMP_NUM_THREADS': '1',
                  'MKL_NUM_THREADS': '1',
                  'OPENBLAS_NUM_THREADS': '1'})

In [3]:

wkDIR = os.getcwd()
outDIR_base = 'data/hls_nrt/'

cluster_loc = 'hpc'

In [4]:
yr = 2024
os.chdir(wkDIR)
#prefix = 'redtop' 
prefix = 'cper'
#prefix = 'snaplands_rr'

full_smooth = True

if prefix == 'cper': 
    aoi_f = os.path.join('data/ground/cper_pastures_2017_clip.shp')
    df_aoi = gpd.read_file(aoi_f)
    subunit_name_old = 'Past_Name_'
    subunit_name = 'Pasture'
elif prefix == 'snaplands_rr':
    aoi_f = os.path.join('data/ground/snaplands_rr_bounded.shp')
    df_aoi = gpd.read_file(aoi_f)
    df_aoi = df_aoi.to_crs(epsg=32613)
    subunit_name_old = None
    subunit_name = None
    aoi_bounds = df_aoi.total_bounds
elif prefix == 'redtop':
    from hlsstack.utils.convert import kmz_to_shp
    df_aoi = kmz_to_shp('data/ground/RedTop_Boundary.kmz', 'data/ground/')
    df_aoi = df_aoi.to_crs(epsg=32613)
    subunit_name_old = None
    subunit_name = None

outDIR = os.path.join(outDIR_base, prefix)

mod_bm = load_model('cper_biomass')
mod_cov = load_model('cper_cover')

In [5]:

if cluster_loc == 'local':
    os.chdir(wkDIR)
    print('   setting up Local cluster...')
    from dask.distributed import LocalCluster, Client
    import dask
    aws=False
    smth_chunks = {'y': 30, 'x': 30, 'time':-1}
    cluster = LocalCluster(n_workers=8, threads_per_worker=2)
    client = Client(cluster)
    display(client)
elif cluster_loc == 'coiled':
    import coiled
    aws=True
    smth_chunks = {'y': 10, 'x': 10, 'time':-1}
    coiled.create_software_environment(
    name="hls_cog_coiled",
    conda="hls_cog_coiled_env.yaml")
    cluster = coiled.Cluster(
        name="hls_cog_coiled",
        software="kearney-sp/hls_cog_coiled",
        n_workers=5,
        worker_cpu=2,
        scheduler_cpu=2,
        backend_options={"region": "us-west-2"},
        environ=dict(GDAL_DISABLE_READDIR_ON_OPEN='FALSE', 
                   #AWS_NO_SIGN_REQUEST='YES',
                   GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
                   GDAL_SWATH_SIZE='200000000',
                   VSI_CURL_CACHE_SIZE='200000000',
                   CPL_VSIL_CURL_ALLOWED_EXTENSIONS='TIF',
                   GDAL_HTTP_UNSAFESSL='YES',
                   GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                   GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'),
                   AWS_REGION='us-west-2',
                   AWS_SECRET_ACCESS_KEY=s3_cred['secretAccessKey'],
                   AWS_ACCESS_KEY_ID=s3_cred['accessKeyId'],
                   AWS_SESSION_TOKEN=s3_cred['sessionToken'])
    )
    client = Client(cluster)
    display(client)
elif cluster_loc == 'hpc':
    from dask.distributed import LocalCluster, Client
    import dask_jobqueue as jq
    import dask
    from jupyter_server import serverapp
    wkDIR = '/project/cper_neon_aop/hls_nrt/'
    outDIR = '/90daydata/cper_neon_aop/hls_nrt/' + prefix
    os.chdir(wkDIR)
    # get the server address for porting
    try:
        jupServer = [x for x in serverapp.list_running_servers()][0]
    except IndexError:
        # manually copy/paste the server address
        jupServer = {'base_url': '/node/ceres19-compute-98-eth.scinet.local/17710/'}
    print('   setting up moderate cluster on HPC...')
    aws=False
    smth_chunks = {'y': 10, 'x': 10, 'time':-1}
    dask.config.set({'distributed.dashboard.link': jupServer['base_url'] + 'proxy/{port}/status'})
    partition='brief-low',#'short','debug', 'mem', 'mem-low',
    num_processes = 4
    num_threads_per_processes = 2
    mem = 2.5*num_processes*num_threads_per_processes
    n_cores_per_job = num_processes*num_threads_per_processes
    clust = jq.SLURMCluster(queue=partition,
                            processes=num_processes,
                            cores=n_cores_per_job,
                            memory=str(mem)+'GB',
                            #interface='ib0',
                            interface='ens7f0',
                            # interface='enp24s0f0',
                            local_directory='$TMPDIR',
                            death_timeout=30,
                            walltime='02:00:00',
                            job_extra=["--output=/dev/null","--error=/dev/null"])
    client=Client(clust)
    #Scale Cluster 
    num_jobs=20
    clust.scale(jobs=num_jobs)
    try:
        client.wait_for_workers(n_workers=num_jobs*num_processes, timeout=60)
    except dask.distributed.TimeoutError as e:
        print(str(num_jobs*num_processes) + ' workers not available. Continuing with available workers.')
        #print(e)
        pass
    display(client)

if not os.path.exists(outDIR):
    os.mkdir(outDIR)

if subunit_name_old is not None:
    df_aoi = df_aoi.rename(columns={subunit_name_old: subunit_name})
if subunit_name is not None:
    df_aoi = df_aoi.dissolve(by=subunit_name).reset_index()

start_date = str(yr - 1) + "-11-01"
end_date = str(yr + 1) + "-03-01"

# set the date range for analysis
date_rng = pd.date_range(start=start_date, end=end_date)
date_rng = date_rng[date_rng <= datetime.today()]

/project/cper_neon_aop/conda_envs/hls_nrt_env/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


   setting up moderate cluster on HPC...


/project/cper_neon_aop/conda_envs/hls_nrt_env/lib/python3.9/site-packages/distributed/node.py:177: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41825 instead
  warnings.warn(


<Client: 'tcp://10.1.5.124:44675' processes=80 threads=160, memory=372.80 GiB>

In [6]:
t00 = time.time()
nc_flist = list(filter(os.path.isfile, 
                    glob.glob(os.path.join(outDIR,
                              prefix + 
                              '_hls_tmp/' + 'hls_ds_' + str(yr) + '*.nc'))))
if len(nc_flist) > 0:
    nc_flist.sort(key=lambda x: os.path.getctime(x))
    nc_f = nc_flist[-1]
else:
    print('ERROR: No temp nc file found')

hls_ds = xr.open_dataset(nc_f).load()
#hls_atsa = xr.open_dataset(re.sub('hls_ds_', 'hls_atsa_', nc_f))

In [7]:
hls_ds

<xarray.Dataset>
Dimensions:         (time: 68, y: 323, x: 323)
Coordinates:
  * time            (time) datetime64[ns] 2023-11-01 2023-11-08 ... 2024-03-18
    id              (time) object 'HLS.L30.T13TEF.2023305T173702.v2.0' ... 'H...
  * x               (x) float64 5.176e+05 5.176e+05 ... 5.272e+05 5.272e+05
  * y               (y) float64 4.524e+06 4.524e+06 ... 4.515e+06 4.515e+06
    eo:cloud_cover  (time) int64 98 93 0 0 26 100 0 ... 66 16 100 100 100 10 1
    epsg            int64 32613
    tile_id         (time) object 'T13TEF' 'T13TEF' ... 'T13TEF' 'T13TEF'
Data variables:
    BLUE            (time, y, x) float64 5.192e+03 5.182e+03 ... 718.0 684.0
    GREEN           (time, y, x) float64 4.997e+03 5.072e+03 ... 992.0 948.0
    RED             (time, y, x) float64 5.067e+03 5.114e+03 ... 1.271e+03
    NIR1            (time, y, x) float64 5.161e+03 5.188e+03 ... 1.895e+03
    SWIR1           (time, y, x) float64 2.068e+03 2.097e+03 ... 3.019e+03
    SWIR2           (time, y, x) float64 2.373e+03 2.392e+03 ... 2.275e+03
    FMASK           (time, y, x) float64 194.0 194.0 194.0 ... 128.0 128.0 128.0
    SZA             (time, y, x) float64 5.741e+03 5.741e+03 ... 4.459e+03
    SAA             (time, y, x) float64 1.611e+04 1.611e+04 ... 1.532e+04
    VZA             (time, y, x) float64 548.0 548.0 547.0 ... 966.0 966.0 966.0
    VAA             (time, y, x) float64 1.001e+04 1.001e+04 ... 9.531e+03
    maskcov_pct     (time) float64 100.0 96.47 0.0 0.0 ... 100.0 0.3738 7.708
Attributes:
    crs:         epsg:32613
    transform:   [ 3.00000e+01  0.00000e+00  5.17590e+05  0.00000e+00 -3.0000...
    resolution:  30

In [8]:
if len(hls_ds['time'].groupby('tile_id').apply(lambda x: x.drop_duplicates('time', False))) < len(hls_ds['time']):
    print('    reducing along id dimension to single observation for each date, keeping least-masked image')
    print('NOTE: THIS SHOULD NOT BE NEEDED! GO BACK AND CHECK HLS2_aoi_yr_download.ipynb')
    hls_ds = hls_ds.groupby('tile_id').apply(lambda x: x.sortby('maskcov_pct').drop_duplicates('time', keep='first')).sortby('time').compute()

In [9]:
#hls_bolton_mask = bolton_mask_xr(hls_ds).compute()
#hls_mask_snow = mask_hls(hls_ds['FMASK'], mask_types=['snow'])
#hls_mask = mask_hls(hls_ds['FMASK'], mask_types=['all'])

#show((hls_bolton.sum(dim='time') / len(hls_mask.time)) > 0.1)

In [10]:
if len(np.unique(hls_ds.tile_id)) > 1:
    hls_atsa = hls_ds.groupby('tile_id').apply(lambda x: atsa_mask(x.where(
        x['BLUE'].notnull(), drop=True))).compute()
    hls_atsa = hls_atsa.transpose('time', 'y', 'x')
else:
    hls_ds = hls_ds.reset_coords(drop=True)
    try:
        hls_atsa = atsa_mask(hls_ds).compute()
        mask_atsa = True
    except ValueError or IndexError:
        print('WARNING: Could not compute ATSA cloud/shadow mask')
        mask_atsa = False
        pass

/project/cper_neon_aop/conda_envs/hls_nrt_env/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


HOT centers of clear, thin and thick cloudy pixels on land : [ 341.5185372  1548.15205992 3044.57681578]
Shadow index centers of shadow, nonshadow dark and nonshadow bright on land : [ 312.08770251 1298.37801922 1762.6174002 ]


In [11]:
client.restart()

<Client: 'tcp://10.1.5.124:44675' processes=51 threads=102, memory=237.66 GiB>

In [12]:
hls_ds = xr.merge([hls_ds, hls_atsa], join='inner')
#hls_mask = mask_hls(hls_ds['FMASK'], mask_types=['all'])
hls_mask = mask_hls(hls_ds['FMASK'], mask_types=['cirrus',
                                                'cloud',
                                                'cloud_adj',
                                                'shadow', 
                                                'snow',
                                                'water'])
#hls_mask = mask_hls(hls_ds['FMASK'], 'all')
hls_ds = hls_ds.where(hls_mask == 0)

In [13]:
if mask_atsa:
    print('masking out clouds and shadows detected by ATSA')
    hls_ds = hls_ds.where(hls_ds['ATSA'] == 1)

masking out clouds and shadows detected by ATSA


In [14]:
if 'tile_id' in hls_ds.coords and len(hls_ds.tile_id) > 1:
    hls_ds = hls_ds.groupby('time').mean()

In [15]:
t0 = time.time()
#print('   applying tertiary cloud mask...')
#hls_ds = hls_ds.chunk({'time': -1, 'y': 100, 'x': 100})
t1 = time.time()
#hls_bolton_mask = bolton_mask_xr(hls_ds).compute()
#hls_ds = hls_ds.where(hls_bolton_mask == 0, drop=True)
# mask out scenes where < 75% of the region is cloud-free
partial_mask = (hls_ds['NIR1'].isnull().sum(dim=['y', 'x'])/np.product(hls_ds['NIR1'].shape[1:])) < 0.25
partial_mask = partial_mask#.persist()
hls_ds = hls_ds.where(partial_mask, drop=True).compute()
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

...completed in 0.0 secs
total elasped time: 0.01 mins



In [16]:
hls_ds

<xarray.Dataset>
Dimensions:      (time: 26, y: 323, x: 323)
Coordinates:
  * time         (time) datetime64[ns] 2023-11-09 2023-11-12 ... 2024-03-18
  * x            (x) float64 5.176e+05 5.176e+05 ... 5.272e+05 5.272e+05
  * y            (y) float64 4.524e+06 4.524e+06 ... 4.515e+06 4.515e+06
Data variables: (12/13)
    BLUE         (time, y, x) float64 893.0 939.0 858.0 ... 734.0 718.0 684.0
    GREEN        (time, y, x) float64 1.219e+03 1.311e+03 ... 992.0 948.0
    RED          (time, y, x) float64 1.662e+03 1.746e+03 ... 1.271e+03
    NIR1         (time, y, x) float64 2.474e+03 2.54e+03 ... 1.993e+03 1.895e+03
    SWIR1        (time, y, x) float64 3.305e+03 3.405e+03 ... 3.019e+03
    SWIR2        (time, y, x) float64 2.553e+03 2.647e+03 ... 2.275e+03
    ...           ...
    SZA          (time, y, x) float64 5.895e+03 5.895e+03 ... 4.459e+03
    SAA          (time, y, x) float64 1.66e+04 1.66e+04 ... 1.532e+04 1.532e+04
    VZA          (time, y, x) float64 1.051e+03 1.05e+03 ... 966.0 966.0
    VAA          (time, y, x) float64 1.108e+04 1.108e+04 ... 9.531e+03
    maskcov_pct  (time, y, x) float64 0.0 0.0 0.0 0.0 ... 7.708 7.708 7.708
    ATSA         (time, y, x) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
Attributes:
    crs:         epsg:32613
    transform:   [ 3.00000e+01  0.00000e+00  5.17590e+05  0.00000e+00 -3.0000...
    resolution:  30

In [17]:
client.restart()

<Client: 'tcp://10.1.5.124:44675' processes=46 threads=92, memory=214.36 GiB>

In [18]:
if not os.path.exists(os.path.join(outDIR, 'hls_ndvi/')):
    os.mkdir(os.path.join(outDIR, 'hls_ndvi/'))
if not os.path.exists(os.path.join(outDIR, 'hls_biomass/')):
    os.mkdir(os.path.join(outDIR, 'hls_biomass/'))
if not os.path.exists(os.path.join(outDIR, 'hls_cover/')):
    os.mkdir(os.path.join(outDIR, 'hls_cover/'))

In [19]:
hls_ds = hls_ds.chunk({'time': 1, 'y': -1, 'x': -1})
print('   computing NDVI for available dates...')
# create ndvi array
xr_ndvi = ndvi_func(hls_ds)#.compute()
xr_ndvi.name = 'NDVI'


print('   creating daily template for output...')
# reset date range to be within dataset
date_rng = date_rng[date_rng <= xr_ndvi['time'].max().values]

# create an output Dataset template with all dates
dat_out = xr.Dataset(coords={'time': [x for x in date_rng if x not in xr_ndvi['time'].values],
                             'x': xr_ndvi.x,
                             'y': xr_ndvi.y})

print('   interpolating and saving to disk')
t1 = time.time()

dat_out_nans = np.zeros((dat_out.dims['time'], 
                         dat_out.dims['y'], 
                         dat_out.dims['x'])) * np.nan

dat_out = dat_out.assign(NDVI=(['time', 'y', 'x'],
                               dat_out_nans))

dat_out_ds = xr.concat([dat_out['NDVI'], xr_ndvi], dim='time').to_dataset()
dat_out_ds = dat_out_ds.sortby('time')
dat_out_ds = dat_out_ds.rio.write_crs(CRS.from_dict(init='epsg:32613'))

yr_mask = dat_out_ds['time'].astype(np.datetime64).dt.year == yr

dat_out_ds = dat_out_ds.chunk({'time': -1, 'y': 20, 'x': 20})

if full_smooth:
    dat_out_da = despike_ts_xr(dat_out_ds['NDVI'],
                               dat_thresh=0.07, 
                               mask_outliers=False,
                               iters=2,
                               dims=['time'])
    dat_out_da = smooth_xr(dat_out_da, 
                           dims=['time'], 
                           kwargs={'double': True, 'limit': 91})
else:
    dat_out_da = dat_out_ds['NDVI'].interpolate_na('time').rolling(time=21, 
                                                                   center=True,
                                                                   min_periods=1).mean()

dat_out_da = dat_out_da.sel(time=yr_mask).astype('float32').rio.reproject("EPSG:3857").compute()

if os.path.exists(os.path.join(outDIR, 'hls_ndvi/' + prefix + '_hls_ndvi_' + str(yr) + '.nc')):
    os.remove(os.path.join(outDIR, 'hls_ndvi/' + prefix + '_hls_ndvi_' + str(yr) + '.nc'))

dat_out_da.to_netcdf(
    os.path.join(outDIR, 'hls_ndvi/' + prefix + '_hls_ndvi_' + str(yr) + '.nc'))

# set permissions to be read/write/execute for all users
os.chmod(os.path.join(outDIR, 'hls_ndvi/' + prefix + '_hls_ndvi_' + str(yr) + '.nc'), 0o777)

print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

del dat_out_da, dat_out_ds, dat_out, dat_out_nans
gc.collect()
client.restart()

   computing NDVI for available dates...
   creating daily template for output...
   interpolating and saving to disk
...completed in 51.0 secs
total elasped time: 1.21 mins



<Client: 'tcp://10.1.5.124:44675' processes=68 threads=136, memory=316.88 GiB>

In [20]:
print('   computing biomass for available dates...')
# create biomass array
xr_bm = hls_ds.map_blocks(pred_bm, template=hls_ds['NIR1'],
                          kwargs=dict(model=mod_bm)).where(hls_ds['NIR1'].notnull())#.compute()
xr_bm.name = 'Biomass'

print('   creating daily template for output...')
# reset date range to be within dataset
date_rng = date_rng[date_rng <= xr_bm['time'].max().values]

# create an output Dataset template with all dates
dat_out = xr.Dataset(coords={'time': [x for x in date_rng if x not in xr_bm['time'].values],
                             'x': xr_bm.x,
                             'y': xr_bm.y})

print('   interpolating and saving to disk')
t1 = time.time()

dat_out_nans = np.zeros((dat_out.dims['time'], 
                         dat_out.dims['y'], 
                         dat_out.dims['x'])) * np.nan

dat_out = dat_out.assign(Biomass=(['time', 'y', 'x'],
                               dat_out_nans))

dat_out_ds = xr.concat([dat_out['Biomass'], xr_bm], dim='time').to_dataset()
dat_out_ds = dat_out_ds.sortby('time')
dat_out_ds = dat_out_ds.rio.write_crs(CRS.from_dict(init='epsg:32613'))

dat_out_ds['Biomass'] = xr.concat([dat_out['Biomass'], xr_bm], dim='time')
dat_out_ds = dat_out_ds.sortby('time').chunk({'time': -1, 'y': 20, 'x': 20})

if full_smooth:
    dat_out_da = despike_ts_xr(dat_out_ds['Biomass'],
                               dat_thresh=150, 
                               mask_outliers=True,
                               z_thresh=5.0,
                               iters=2,
                               dims=['time'])
    dat_out_da = smooth_xr(dat_out_da, 
                           dims=['time'], 
                           kwargs={'double': True, 'limit': 91})
else:
    dat_out_da = dat_out_ds['Biomass'].interpolate_na('time').rolling(time=21, 
                                                                      center=True,
                                                                      min_periods=1).mean()

dat_out_da = dat_out_da.sel(time=yr_mask).astype('float32').rio.reproject("EPSG:3857").compute()

# delete output if it already exists
if os.path.exists(os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_' + str(yr) + '.nc')):
    os.remove(os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_' + str(yr) + '.nc'))
# save output to disk
dat_out_da.to_netcdf(
    os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_' + str(yr) + '.nc'))
# set permissions to be read/write/execute for all users
os.chmod(os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_' + str(yr) + '.nc'), 0o777)

print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

del dat_out_da, dat_out_ds, dat_out, dat_out_nans
gc.collect()
client.restart()

   computing biomass for available dates...
   creating daily template for output...
   interpolating and saving to disk
...completed in 71.0 secs
total elasped time: 2.76 mins



<Client: 'tcp://10.1.5.124:44675' processes=73 threads=146, memory=340.18 GiB>

In [21]:
print('   computing biomass SE for available dates...')
# create biomass SE array
xr_bm_se = hls_ds.map_blocks(pred_bm_se, template=hls_ds['NIR1'],
                          kwargs=dict(model=mod_bm)).where(hls_ds['NIR1'].notnull())#.compute()
xr_bm_se.name = 'Biomass_SE'

print('   creating daily template for output...')
# reset date range to be within dataset
date_rng = date_rng[date_rng <= xr_bm_se['time'].max().values]

# create an output Dataset template with all dates
dat_out = xr.Dataset(coords={'time': [x for x in date_rng if x not in xr_bm_se['time'].values],
                             'x': xr_bm_se.x,
                             'y': xr_bm_se.y})

print('   interpolating and saving to disk')
t1 = time.time()

dat_out_nans = np.zeros((dat_out.dims['time'], 
                         dat_out.dims['y'], 
                         dat_out.dims['x'])) * np.nan

dat_out = dat_out.assign(Biomass_SE=(['time', 'y', 'x'],
                               dat_out_nans))

dat_out_ds = xr.concat([dat_out['Biomass_SE'], xr_bm_se], dim='time').to_dataset()
dat_out_ds = dat_out_ds.sortby('time')
dat_out_ds = dat_out_ds.rio.write_crs(CRS.from_dict(init='epsg:32613'))

dat_out_ds['Biomass_SE'] = xr.concat([dat_out['Biomass_SE'], xr_bm_se], dim='time')
dat_out_ds = dat_out_ds.sortby('time').chunk({'time': -1, 'y': 20, 'x': 20})

dat_out_da = dat_out_ds['Biomass_SE'].interpolate_na('time').rolling(time=21, 
                                                                     center=True,
                                                                     min_periods=1).mean()

dat_out_da = dat_out_da.sel(time=yr_mask).astype('float32').rio.reproject("EPSG:3857").compute()

# delete output if it already exists
if os.path.exists(os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_se_' + str(yr) + '.nc')):
    os.remove(os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_se_' + str(yr) + '.nc'))
# save output to disk
dat_out_da.to_netcdf(
    os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_se_' + str(yr) + '.nc'))
# set permissions to be read/write/execute for all users
os.chmod(os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_se_' + str(yr) + '.nc'), 0o777)

print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

del dat_out_da, dat_out_ds, dat_out, dat_out_nans
gc.collect()
client.restart()

   computing biomass SE for available dates...
   creating daily template for output...
   interpolating and saving to disk
...completed in 55.0 secs
total elasped time: 4.09 mins



<Client: 'tcp://10.1.5.124:44675' processes=60 threads=120, memory=279.60 GiB>

In [22]:
print('   computing cover for available dates...')
# create cover array
xr_cov = pred_cov(hls_ds, 
                  model=mod_cov).where(hls_ds['NIR1'].notnull()).compute()

print('   creating daily template for output...')
# reset date range to be within dataset
date_rng = date_rng[date_rng <= xr_cov['time'].max().values]

for c in ['BARE', 'SD', 'GREEN', 'LITT']:
    print(c)
    # create an output Dataset template with all dates
    dat_out = xr.Dataset(coords={'time': [x for x in date_rng if x not in xr_cov['time'].values],
                                 'x': xr_cov.x,
                                 'y': xr_cov.y})

    print('   interpolating and saving to disk')
    t1 = time.time()

    dat_out_nans = np.zeros((dat_out.dims['time'], 
                             dat_out.dims['y'], 
                             dat_out.dims['x'])) * np.nan

    dat_out = dat_out.assign(TMP=(['time', 'y', 'x'],
                                   dat_out_nans))
    dat_out = dat_out.rename({'TMP': c})

    dat_out_ds = xr.concat([dat_out[c], xr_cov[c]], dim='time').to_dataset()
    dat_out_ds = dat_out_ds.sortby('time')
    dat_out_ds = dat_out_ds.rio.write_crs(CRS.from_dict(init='epsg:32613'))

    dat_out_ds[c] = xr.concat([dat_out[c], xr_cov[c]], dim='time')
    dat_out_ds = dat_out_ds.sortby('time').chunk({'time': -1, 'y': 20, 'x': 20})
    
    if full_smooth:
        dat_out_da = despike_ts_xr(dat_out_ds[c],
                                   dat_thresh=0.15, 
                                   mask_outliers=False,
                                   iters=2,
                                   dims=['time'])
        dat_out_da = smooth_xr(dat_out_da, 
                               dims=['time'], 
                               kwargs={'double': True, 'limit': 91})
    else:
        dat_out_da = dat_out_ds[c].interpolate_na('time').rolling(time=21, 
                                                                  center=True,
                                                                  min_periods=1).mean()
        
    dat_out_da = dat_out_da.sel(time=yr_mask).astype('float32').rio.reproject("EPSG:3857").compute()
    # delete output if it already exists
    if os.path.exists(os.path.join(outDIR, 'hls_cover/' + prefix + '_hls_' + c + '_' + str(yr) + '.nc')):
        os.remove(os.path.join(outDIR, 'hls_cover/' + prefix + '_hls_' + c + '_' + str(yr) + '.nc'))
    # save output to disk
    dat_out_da.to_netcdf(
        os.path.join(outDIR, 'hls_cover/' + prefix + '_hls_' + c + '_' + str(yr) + '.nc'))
    # set permissions to be read/write/execute for all users
    os.chmod(os.path.join(outDIR, 'hls_cover/' + prefix + '_hls_' + c + '_' + str(yr) + '.nc'), 0o777)

    print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
    print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

    del dat_out_da, dat_out_ds, dat_out, dat_out_nans
    gc.collect()
    client.restart()

   computing cover for available dates...
   creating daily template for output...
BARE
   interpolating and saving to disk
...completed in 30.0 secs
total elasped time: 5.27 mins

SD
   interpolating and saving to disk
...completed in 35.0 secs
total elasped time: 6.18 mins

GREEN
   interpolating and saving to disk
...completed in 36.0 secs
total elasped time: 7.09 mins

LITT
   interpolating and saving to disk
...completed in 33.0 secs
total elasped time: 7.93 mins



In [23]:
hls_ds

<xarray.Dataset>
Dimensions:      (time: 26, y: 323, x: 323)
Coordinates:
  * time         (time) datetime64[ns] 2023-11-09 2023-11-12 ... 2024-03-18
  * x            (x) float64 5.176e+05 5.176e+05 ... 5.272e+05 5.272e+05
  * y            (y) float64 4.524e+06 4.524e+06 ... 4.515e+06 4.515e+06
Data variables: (12/13)
    BLUE         (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    GREEN        (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    RED          (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    NIR1         (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    SWIR1        (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    SWIR2        (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    ...           ...
    SZA          (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    SAA          (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    VZA          (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    VAA          (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    maskcov_pct  (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
    ATSA         (time, y, x) float64 dask.array<chunksize=(1, 323, 323), meta=np.ndarray>
Attributes:
    crs:         epsg:32613
    transform:   [ 3.00000e+01  0.00000e+00  5.17590e+05  0.00000e+00 -3.0000...
    resolution:  30